
# <center><font color= #F08080 > <b>CU50_Distribución geográfica de puntos de carga de vehículos eléctricos</font></center>

***
> # <font color='steelblue'> <b>DEPLOY</font>

# <font color='green'><b>ANÁLISIS DE LA DEMANDA</font>

FUNCIONAMIENTO

- Calcula la DEMANDA asociada a cada tesela de la zona.
- VISUALIZACIÓN: de las demandas calculadas en mapa y análisis.

#### LIBRERÍAS

In [1]:
# Load 
%run -i LIBRERIAS.py

#### VARIABLES DE USUARIO

In [2]:
# Load 
%run -i VARIABLES.py

#### CARGA DE FICHEROS

In [3]:
file_name='TESELAS.geojson'
TESELAS = gpd.GeoDataFrame.from_file(file_name)
TESELAS.geometry.crs = "EPSG:4326"

#### DEMANDA EXISTENTE

In [4]:
# Calculamos la probabilidad de que un VE cargue en una tesela en función del AREA no residencial que tiene, junto con 
# la probabilidad de carga general establecida en los parámetros iniciales
PROBABILIDAD_CARGA_VE = TESELAS["AREA_NO_RESIDENCIAL"] / TESELAS["AREA"] * probabilidad_de_carga

# La metemos en la tesela como columna
TESELAS["PROBABILIDAD_CARGA_VE"] = TESELAS["AREA_NO_RESIDENCIAL"] / TESELAS["AREA"] * probabilidad_de_carga

In [5]:
# Calculamos la demanda existente en cada tesela en base al ratio de uso, el tráfico y la probabilidad de carga
TESELAS["DEMANDA_EXISTENTE"] = ratio_penetracion_VE * TESELAS["PROBABILIDAD_CARGA_VE"] * TESELAS["TRAFICO_INTENSIDAD"]        

Calculamos ratio para verlo mejor

In [6]:
TESELAS['DEMANDA_EXISTENTE_RATIO'] = 0
max=TESELAS['DEMANDA_EXISTENTE'].max()
min=TESELAS['DEMANDA_EXISTENTE'].min()
for i in TESELAS.index:
    TESELAS.at[i,'DEMANDA_EXISTENTE_RATIO']=((TESELAS.loc[i,'DEMANDA_EXISTENTE']-min)/(max-min))*10

#### DEMANDA ABSORBIDA

Calculamos la demanda absorbida por las estaciones existentes. 

La calculamos asumiendo que es la existente que la absorben las estaciones de recarga existentes en la zona.

In [7]:
# Calculamos la demanda absorbida
TESELAS["DEMANDA_ABSORBIDA"] = TESELAS["ESTACIONES_VE"] * TESELAS["DEMANDA_EXISTENTE"]

Calculamos ratio para verlo mejor

In [8]:
TESELAS['DEMANDA_ABSORBIDA_RATIO'] = 0
max=TESELAS['DEMANDA_ABSORBIDA'].max()
min=TESELAS['DEMANDA_ABSORBIDA'].min()
for i in TESELAS.index:
    TESELAS.at[i,'DEMANDA_ABSORBIDA_RATIO']=((TESELAS.loc[i,'DEMANDA_ABSORBIDA']-min)/(max-min))*10

#### DEMANDA FINAL

In [9]:
# Se calcula la demanda final, restando la absorbida de la demanda existente 
# Si el resultado es negativo, se establece en cero ya que no puede haber una demanda negativa. 

# La introduzco en la tesela
TESELAS["DEMANDA_FINAL"] = TESELAS["DEMANDA_EXISTENTE"] - TESELAS["DEMANDA_ABSORBIDA"]
        
# La genero en una variable
for i in TESELAS.index:
    # No puede haber demanda negativa, por lo que se limita la demanda mínima a 0.0005
    if TESELAS.loc[i,"DEMANDA_FINAL"] < 0:       
        TESELAS.at[i,"DEMANDA_FINAL"]=0.0005
    

Calculamos ratio para verlo mejor

In [10]:
TESELAS['DEMANDA_FINAL_RATIO'] = 0
max=TESELAS['DEMANDA_FINAL'].max()
min=TESELAS['DEMANDA_FINAL'].min()
for i in TESELAS.index:
    TESELAS.at[i,'DEMANDA_FINAL_RATIO']=((TESELAS.loc[i,'DEMANDA_FINAL']-min)/(max-min))*10

#### DEMANDA DE LOS CANDIDATOS

Actualizo los CANDIDATOS con las columnas de demanda calculadas.


In [11]:
# Extraigo de TESELA las teselas CANDIDATOS. 
CANDIDATOS=TESELAS.loc[TESELAS['CANDIDATO']>0]

In [12]:
#lista de coordenadas de estacionamiento coords_parking a partir de las columnas 'Long' y 'Lat' 
coords_CANDIDATOS = [(x, y) for x, y in zip(CANDIDATOS['CENTROID_LONG'], CANDIDATOS['CENTROID_LAT'])]

#lista de coordenadas de demanda coords_demand a partir de las columnas 'CENTROID_LONG' y 'CENTROID_LAT'
coords_TESELAS = [(x, y) for x, y in zip(TESELAS['CENTROID_LONG'], TESELAS['CENTROID_LAT'])]

# Calculo de la la matriz de distancias entre cada par de coordenadas de estacionamiento y demanda utilizando la función cdist
distance_matrix = distance.cdist(coords_CANDIDATOS, coords_TESELAS, 'euclidean')
scaling_ratio = 100000 # pasamos a metros. 
distance_matrix2 = scaling_ratio * distance_matrix

# Convierte la matriz de distancias escalada a un DataFrame de pandas con los índices de fila del DataFrame df_parking y 
# los índices de columna del DataFrame df_demand
matriz_distancias = pd.DataFrame(distance_matrix2, index=CANDIDATOS.index, # Aqui metemos los índices correctos en la matriz
                                    columns=TESELAS.index.tolist())

In [13]:
TESELAS['DEMANDA_A_ATENDER'] = 0 
for f in CANDIDATOS.index:
    demand=0
    for c in range(len(matriz_distancias.columns)):
        if matriz_distancias.loc[f,c] <= distancia_andando:
            demand = demand + TESELAS.loc[c,'DEMANDA_FINAL']
    TESELAS.at[f,'DEMANDA_A_ATENDER'] = demand

In [14]:
# Extraigo de TESELA las teselas CANDIDATOS. 
CANDIDATOS=TESELAS.loc[TESELAS['CANDIDATO']>0]

#### VISUALIZACIÓN

MAPA

In [ ]:
Datos=TESELAS # Datos a visualizar
variable='DEMANDA_FINAL_RATIO' # Columna a visualizar
tipo_mapa='carto-positron' # 'open-street-map' 
color= 'matter'  # 'icefire'  'ylorbr' # 'ylorbr'
lat_media=TESELAS['CENTROID_LAT'].mean()
long_media=TESELAS['CENTROID_LONG'].mean()
fig = px.scatter_mapbox(Datos, lat = 'CENTROID_LAT', lon = 'CENTROID_LONG', title='Demanda',
                        #center = dict(lat = 40.45, lon = -3.70), zoom = 10,
                        center = dict(lat=lat_media, lon=long_media), zoom = 10,
                        mapbox_style = tipo_mapa, height=750, color=variable, size=variable,
                        color_continuous_scale = color,
                        hover_data=['CANDIDATO','ESTACIONES_VE','TRAFICO_INTENSIDAD_RATIO',
                                    'PORCENTAJE_NO_RESIDENCIAL','DEMANDA_EXISTENTE_RATIO',
                                    'DEMANDA_ABSORBIDA_RATIO','DEMANDA_A_ATENDER'], 
                        opacity=0.3  ) 
fig.show()

ANÁLISIS

In [16]:
# df = px.data.gapminder() #.query("year == 2007").query("continent == 'Europe'")
# df.loc[df['pop'] < 2.e6, 'country'] = 'Other countries' # Represent only large countries
fig = px.pie(CANDIDATOS, values='DEMANDA_A_ATENDER', names='BARRIO', title='Demanda a atender por barrio')
fig.show()

#### GUARDAR FICHEROS

In [17]:
# Guardar
file_name='TESELAS'
TESELAS.to_file(file_name+'.geojson', driver="GeoJSON")